In [1]:
# Standard imports.
import saspy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from datetime import datetime
from numpy import nan as NA
import math

In [2]:
# Get a SAS session object
sas = saspy.SASsession(cfgname="oda")

SAS Connection established. Subprocess id is 835822



In [3]:
# Assign the PG1 library
sas.saslib('pg1', engine='base', path='~/PG1/Data/data/data')

5                                                          The SAS System                         Friday, March  5, 2021 05:35:00 PM

24         
25         libname pg1 base  '~/PG1/Data/data/data'  ;
26         
27         
28         
6                                                          The SAS System                         Friday, March  5, 2021 05:35:00 PM

29         


---
# Activity: p104a04.sas
```
data storm_length;
	set pg1.storm_summary;
	drop Hem_EW Hem_NS Lat Lon;
	*Add assignment statement;
	StormLength = EndDate - StartDate;
run;
```

In [4]:
# Creat a new DataFrame object by pulling PG1.STORM_SUMMARY data from SAS.
storm_summary_df = sas.sasdata('STORM_SUMMARY', 'PG1').to_df()

In [6]:
# Print out the first 5 records
storm_summary_df.head()

,Season,Name,Basin,Type,MaxWindMPH,MinPressure,StartDate,EndDate,Hem_NS,Hem_EW,Lat,Lon
0,1980,NaN,na,TS,35.0,NaN,1980-07-17,1980-11-18,N,W,25.7,-91.2
1,1980,NaN,SP,NR,NaN,998.0,1980-03-27,1980-03-30,S,E,19.1,137.0
2,1980,AGATHA,EP,TS,115.0,NaN,1980-06-09,1980-06-15,N,W,12.8,-118.7
3,1980,ALBINE,SI,ET,NaN,NaN,1979-11-27,1979-12-06,S,E,19.1,137.0
4,1980,ALEX,WP,TS,40.0,998.0,1980-10-09,1980-10-14,N,E,27.2,140.5


In [7]:
# Drop the columns we don't want
storm_length_df = storm_summary_df.drop(columns=['Hem_EW', 'Hem_NS', 'Lat', 'Lon'])

In [8]:
storm_length_df.head()

,Season,Name,Basin,Type,MaxWindMPH,MinPressure,StartDate,EndDate
0,1980,NaN,na,TS,35.0,NaN,1980-07-17,1980-11-18
1,1980,NaN,SP,NR,NaN,998.0,1980-03-27,1980-03-30
2,1980,AGATHA,EP,TS,115.0,NaN,1980-06-09,1980-06-15
3,1980,ALBINE,SI,ET,NaN,NaN,1979-11-27,1979-12-06
4,1980,ALEX,WP,TS,40.0,998.0,1980-10-09,1980-10-14


In [9]:
storm_length_df.loc[4:9, ['Basin', 'Type', 'Name']]

,Basin,Type,Name
4,WP,TS,ALEX
5,NA,TS,ALLEN
6,SI,NR,AMY
7,SI,TS,BERENICE
8,WP,ET,BETTY
9,EP,TS,BLAS


In [10]:
# Create a new column as the delta of EndDate and StartDate
storm_length_df['StormLength'] = storm_length_df['EndDate'] - storm_length_df['StartDate']
storm_length_df

,Season,Name,Basin,Type,MaxWindMPH,MinPressure,StartDate,EndDate,StormLength
0,1980,NaN,na,TS,35.0,NaN,1980-07-17,1980-11-18,124 days
1,1980,NaN,SP,NR,NaN,998.0,1980-03-27,1980-03-30,3 days
2,1980,AGATHA,EP,TS,115.0,NaN,1980-06-09,1980-06-15,6 days
3,1980,ALBINE,SI,ET,NaN,NaN,1979-11-27,1979-12-06,9 days
4,1980,ALEX,WP,TS,40.0,998.0,1980-10-09,1980-10-14,5 days
...,...,...,...,...,...,...,...,...,...
3113,2016,VARDAH,NI,NR,81.0,975.0,2016-12-06,2016-12-17,11 days
3114,2016,VICTOR,SP,NR,92.0,958.0,2016-01-14,2016-01-22,8 days
3115,2016,WINSTON,SP,NR,173.0,884.0,2016-02-10,2016-02-26,16 days
3116,2016,YALO,SP,NR,46.0,100.0,2016-02-24,2016-02-26,2 days


---
# Activity: p104a05.sas
```
data storm_wingavg;
	set pg1.storm_range;
	*Add assignment statements;
	WindAve = mean(Wind1, Wind2, Wind3, Wind4);
	WindRange = range(of Wind1-Wind4);
run;
```

In [11]:
# Creat a new DataFrame object by pulling PG1.STORM_SUMMARY data from SAS.
storm_range_df = sas.sasdata('STORM_RANGE', 'PG1').to_df()

In [12]:
# For convience create a new DataFrame with only the Wind 1-4 columns
wdf = storm_range_df.loc[:, ['Wind1', 'Wind2', 'Wind3', 'Wind4']]

In [13]:
wdf

,Wind1,Wind2,Wind3,Wind4
0,100,95,90,85
1,50,50,50,45
2,65,65,65,65
3,45,45,35,30
4,40,35,35,25
...,...,...,...,...
2954,105,105,105,105
2955,60,60,60,60
2956,95,85,80,75
2957,100,100,95,95


In [37]:
# Add a new column for the mean of Wind 1-4
storm_range_df['WindAve'] = wdf.mean(axis=1)

,Season,Basin,Name,Wind1,Wind2,Wind3,Wind4,WindAve
0,1980,EP,AGATHA,100,95,90,85,92.50
1,1980,EP,BLAS,50,50,50,45,48.75
2,1980,EP,CELIA,65,65,65,65,65.00
3,1980,EP,DARBY,45,45,35,30,38.75
4,1980,EP,ESTELLE,40,35,35,25,33.75
...,...,...,...,...,...,...,...,...
2954,2016,WP,NOCK-TEN,105,105,105,105,105.00
2955,2016,WP,OMAIS,60,60,60,60,60.00
2956,2016,WP,SARIKA,95,85,80,75,83.75
2957,2016,WP,SONGDA,100,100,95,95,97.50


In [15]:
# Add a new column for the range of Wind 1-4
storm_range_df['WindRange'] = wdf.loc[0:].max(axis=1) - wdf.loc[0:].min(axis=1)

In [40]:
storm_range_df.head(5)

,Season,Basin,Name,Wind1,Wind2,Wind3,Wind4,WindAve,WindRange
0,1980,EP,AGATHA,100,95,90,85,92.50,15
1,1980,EP,BLAS,50,50,50,45,48.75,5
2,1980,EP,CELIA,65,65,65,65,65.00,0
3,1980,EP,DARBY,45,45,35,30,38.75,15
4,1980,EP,ESTELLE,40,35,35,25,33.75,15


---
# Activity: p104a06.sas
```
data pacific;
	set pg1.storm_summary;
	drop Type Hem_EW Hem_NS MinPressure Lat Lon;
	*Add a WHERE statement that uses the SUBSTR function;
	where substr(basin, 2, 1) eq 'P';
run;

```

In [16]:
# Drop the columns we don't want
pacific_df = storm_summary_df.drop(columns=['Hem_EW', 'Hem_NS', 'MinPressure', 'Lat', 'Lon'])

In [17]:
# Filter out the rows where the Basin named 
pacific_df = pacific_df[list(map(lambda b: b[1:2] == 'P', pacific_df['Basin']))]

In [18]:
pacific_df

,Season,Name,Basin,Type,MaxWindMPH,StartDate,EndDate
1,1980,NaN,SP,NR,NaN,1980-03-27,1980-03-30
2,1980,AGATHA,EP,TS,115.0,1980-06-09,1980-06-15
4,1980,ALEX,WP,TS,40.0,1980-10-09,1980-10-14
8,1980,BETTY,WP,ET,115.0,1980-10-28,1980-11-08
9,1980,BLAS,EP,TS,58.0,1980-06-16,1980-06-19
...,...,...,...,...,...,...,...
3111,2016,ULIKA,EP,DS,75.0,2016-09-25,2016-10-03
3114,2016,VICTOR,SP,NR,92.0,2016-01-14,2016-01-22
3115,2016,WINSTON,SP,NR,173.0,2016-02-10,2016-02-26
3116,2016,YALO,SP,NR,46.0,2016-02-24,2016-02-26


---
# Activity: p104a07.sas
```
data storm_cat;
	set pg1.storm_summary;
	keep Name Basin MinPressure StartDate PressureGroup;
	*add ELSE keyword and remove final condition;
	if MinPressure=. then PressureGroup=.;
	else if MinPressure<=920 then PressureGroup=1;
	else PressureGroup=0;
	
run;

proc freq data=storm_cat;
	tables PressureGroup;
run;
```

In [19]:
# Python doesn't have a 'keep' function so we need to stick with 'drop'.
# For convienece I create a DropList and pass it to the drop() method.
DropList = ['EndDate', 'Hem_EW', 'Hem_NS', 'Lat', 'Lon', 'MaxWindMPH', 'Season', 'Type']
storm_cat_df = storm_summary_df.drop(columns=DropList)

In [20]:
for i in storm_cat_df.index: 
    mp = storm_cat_df.loc[i, 'MinPressure']
    if math.isnan(mp): 
        pg = NA
    elif mp <= 920:
        pg = 1
    else:
        pg = 0
    storm_cat_df.loc[i, 'PressureGroup'] = pg

In [21]:
storm_cat_df

,Name,Basin,MinPressure,StartDate,PressureGroup
0,NaN,na,NaN,1980-07-17,NaN
1,NaN,SP,998.0,1980-03-27,0.0
2,AGATHA,EP,NaN,1980-06-09,NaN
3,ALBINE,SI,NaN,1979-11-27,NaN
4,ALEX,WP,998.0,1980-10-09,0.0
...,...,...,...,...,...
3113,VARDAH,NI,975.0,2016-12-06,0.0
3114,VICTOR,SP,958.0,2016-01-14,0.0
3115,WINSTON,SP,884.0,2016-02-10,1.0
3116,YALO,SP,100.0,2016-02-24,1.0


In [24]:
# Get a frequency report
storm_cat_df['PressureGroup'].value_counts()

0.0    2733
1.0     189
Name: PressureGroup, dtype: int64